# 양방향  LSTM +  CRF 이용 개체명 인식

##  CRF layer 사용을 위한 환경구성

### tensorflow 1.13.1 install

In [ ]:
!pip install tensorflow==1.13.1

### keras 2.2.4 install

In [ ]:
!pip install keras==2.2.4

### keras contrib( CRF layer ) install

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

### 개체명 인식을 위한 데이터 이해 및 전처리 

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
data = pd.read.csv( 'ner_dataset.csv', encoding = 'latin1' )

In [ ]:
data[ :5 ]

In [ ]:
print( '데이터프레임 행의 개수 : {}'.format( len( data ) ) )

In [ ]:
print( '데이터에 null 값이 있는지 유무 : {}'.format( str( data.isnull().values.any() ) ) )

In [ ]:
print( '어떤 열에 null 값이 있는지 출력' )
print( '=================================' )
data.isnull().sum()

In [ ]:
print( 'sentence # 열의 중복을 제거한 값의 개수 : {}'.format( data[ 'Sentence #' ].nunique() ) )
print( 'Word 열의 중복을 제거한 값의 개수 : {}'.format( data.Word.nunique() ) )
print( 'Tag 열의 중복을 제거한 값의 개수 : {}'.format( data.Tag.nunique() ) )

In [ ]:
print( 'Tag 열의 각각의 값의 개수 카운트' )
print( '================================' )
print( data.groupby( 'Tag' ).size().reset_index( name = 'count' ) )

In [ ]:
data = data.fillna( method = "ffill" )

In [ ]:
print( data.tail() )

In [ ]:
print( '데이터에 Null 값이 있는지 유무 : ' + str( data.isnull().values.any() ) )

In [ ]:
data[ 'Word' ] = data[ 'Word' ].str.lower()
print( 'Word 열의 중복을 제거한 값의 개수 : {}'.format( data.Word.nunique() ) )

In [ ]:
print( data[ :5 ] )

In [ ]:
func = lambda temp : [ ( w, t ) for w, t in zip( temp[ "Word" ].values.tolist(), temp[ "Tag" ].values.tolist() ) ]
tagged_sentences = [ t for t in data.groupby( "Sentence #" ).apply( func ) ]
print( "전체 샘플 개수: {}".format( len( tagged_sentences ) ) )

In [ ]:
print( tagged_sentences[ 0 ] )

In [ ]:
sentences, ner_tags = [], [] 
for tagged_sentence in tagged_sentences: # 47,959개의 문장 샘플을 1개씩 불러온다.
    sentence, tag_info = zip( *tagged_sentence ) # 각 샘플에서 단어들은 sentence에 개체명 태깅 정보들은 tag_info에 저장.
    sentences.append( list( sentence ) ) # 각 샘플에서 단어 정보만 저장한다.
    ner_tags.append( list( tag_info ) ) # 각 샘플에서 개체명 태깅 정보만 저장한다.

In [ ]:
print( sentences[ 0 ] )
print( ner_tags[ 0 ] )

In [ ]:
print( sentences[ 98 ] )
print( ner_tags[ 98 ] )

In [ ]:
print( '샘플의 최대 길이 : %d' % max( len( l ) for l in sentences ) )
print( '샘플의 평균 길이 : %f' % ( sum( map( len, sentences ) ) / len( sentences ) ) )
plt.hist( [ len( s ) for s in sentences] , bins = 50 )
plt.xlabel( 'length of samples' )
plt.ylabel( 'number of samples' )
plt.show()

### Tokenization

In [ ]:
src_tokenizer = Tokenizer( oov_token = 'OOV' ) # 모든 단어를 사용하지만 인덱스 1에는 단어 'OOV'를 할당한다.
src_tokenizer.fit_on_texts( sentences )
tar_tokenizer = Tokenizer( lower = False ) # 태깅 정보들은 내부적으로 대문자를 유지한채로 저장
tar_tokenizer.fit_on_texts( ner_tags )

In [ ]:
vocab_size = len( src_tokenizer.word_index ) + 1
tag_size = len( tar_tokenizer.word_index ) + 1
print( '단어 집합의 크기 : {}'.format( vocab_size ) )
print( '개체명 태깅 정보 집합의 크기 : {}'.format( tag_size ) )

In [ ]:
print( '단어 OOV의 인덱스 : {}'.format( src_tokenizer.word_index[ 'OOV' ] ) )

### word representation

In [ ]:
X_train = src_tokenizer.texts_to_sequences( sentences )
y_train = tar_tokenizer.texts_to_sequences( ner_tags )

In [ ]:
print( X_train[ 0 ] )
print( y_train[ 0 ] )

In [ ]:
index_to_word = src_tokenizer.index_word
index_to_ner = tar_tokenizer.index_word
index_to_ner[ 0 ] = 'PAD'

In [ ]:
print( index_to_ner )

In [ ]:
decoded = []
for index in X_train[ 0 ] : # 첫번째 샘플 안의 인덱스들에 대해서
    decoded.append( index_to_word[ index ] ) # 다시 단어로 변환

print( '기존의 문장 : {}'.format( sentences[ 0 ] ) )
print( '디코딩 문장 : {}'.format( decoded ) )

In [ ]:
max_len = 70
X_train = pad_sequences( X_train, padding = 'post', maxlen = max_len )
# X_train의 모든 샘플들의 길이를 맞출 때 뒤의 공간에 숫자 0으로 채움.

y_train = pad_sequences( y_train, padding = 'post', maxlen = max_len )
# y_train의 모든 샘플들의 길이를 맞출 때 뒤의 공간에 숫자0으로 채움.

### 훈련/테스트 데이터

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X_train, y_train, test_size = .2, random_state = 777 )

In [ ]:
y_train = to_categorical( y_train, num_classes = tag_size )
y_test = to_categorical( y_test, num_classes = tag_size )

In [ ]:
print( '훈련 샘플 문장의 크기 : {}'.format( X_train.shape ) )
print( '훈련 샘플 레이블의 크기 : {}'.format( y_train.shape ) )
print( '테스트 샘플 문장의 크기 : {}'.format( X_test.shape ) )
print( '테스트 샘플 레이블의 크기 : {}'.format( y_test.shape ) )

### 양방향 LSTM + CRF를 이용한 개체명 인식 모델

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [ ]:
model = Sequential()
model.add( Embedding( input_dim = vocab_size, output_dim = 20, input_length = max_len, mask_zero = True ) )
model.add( Bidirectional( LSTM( units = 50, return_sequences = True, recurrent_dropout = 0.1 ) ) )
model.add( TimeDistributed( Dense( 50, activation = "relu" ) ) )
crf = CRF( tag_size )
model.add( crf )

In [ ]:
model.compile( optimizer = "rmsprop", loss = crf.loss_function, metrics = [ crf.accuracy ] )
history = model.fit( X_train, y_train, batch_size = 32, epochs = 5, validation_split = 0.1, verbose = 1 )

In [ ]:
print( "\n 테스트 정확도: %.4f" % ( model.evaluate( X_test, y_test )[ 1 ] ) )

In [ ]:
i=13 # 확인하고 싶은 테스트용 샘플의 인덱스.
y_predicted = model.predict( np.array( [ X_test[ i ] ] ) ) # 입력한 테스트용 샘플에 대해서 예측 y를 리턴
y_predicted = np.argmax( y_predicted, axis = -1 ) # 원-핫 인코딩을 다시 정수 인코딩으로 변경함.
true = np.argmax( y_test[ i ], -1 ) # 원-핫 인코딩을 다시 정수 인코딩으로 변경함.

print( "{:15}|{:5}|{}".format( "단어", "실제값", "예측값" ) )
print( 35 * "-")

for w, t, pred in zip( X_test[ i ], true, y_predicted[ 0 ]) :
    if w != 0: # PAD값은 제외함.
        print( "{:17}: {:7} {}".format( index_to_word[ w ], index_to_ner[ t ], index_to_ner[ pred ] ) )

In [ ]:
epochs = range( 1, len(history.history[ 'val_loss' ] ) + 1 )
plt.plot( epochs, history.history[ 'loss' ] )
plt.plot( epochs, history.history[ 'val_loss'] )
plt.title( 'model loss' )
plt.ylabel( 'loss' )
plt.xlabel( 'epoch' )
plt.legend( [ 'train', 'val' ], loc = 'upper left' )
plt.show()

### f1 score install

In [ ]:
!pip install seqeval

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

### 훈련된 모델  f1 score

In [ ]:
def sequences_to_tag( sequences ): # 예측값을 index_to_tag를 사용하여 태깅 정보로 변경하는 함수.
    result = []
    for sequence in sequences: # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence: # 시퀀스로부터 예측값을 하나씩 꺼낸다.
            pred_index = np.argmax( pred ) # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            temp.append( index_to_ner[ pred_index ].replace( "PAD", "O" ) ) # 'PAD'는 'O'로 변경
        result.append( temp )
    return result

In [ ]:
y_predicted = model.predict( X_test )
pred_tags = sequences_to_tag( y_predicted )
test_tags = sequences_to_tag( y_test )

In [ ]:
print( classification_report( test_tags, pred_tags ) )

In [ ]:
print( "F1-score: {:.1%}".format( f1_score( test_tags, pred_tags ) ) )

### 새로운 입력에 대한 예측

In [ ]:
word_to_index = src_tokenizer.word_index
new_sentence = 'Hong Gildong starts over with the Justice League of Joseon at the country of Yul'.split()

In [ ]:
new_X = []
for w in new_sentence:
    try:
      new_X.append( word_to_index.get( w, 1 ) )
    except KeyError:
      new_X.append( word_to_index[ 'OOV' ] )
      # 모델이 모르는 단어에 대해서는 'OOV'의 인덱스인 1로 인코딩

print( new_X )

In [ ]:
pad_new = pad_sequences( [ new_X ], padding = "post", value = 0, maxlen = max_len )

In [ ]:
p = model.predict( np.array( [ pad_new[ 0 ] ] ) )
p = np.argmax( p, axis = -1 )
print( "{:15}||{}".format( "단어", "예측값" ) )
print( 30 * "=" )
for w, pred in zip( new_sentence, p[ 0 ] ):
    print( "{:15}: {:5}".format( w, index_to_ner[ pred ] ) )